In [1]:
import pickle
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from spellchecker import SpellChecker

# Load your dialog dataset
# Replace 'ab.csv' with the actual path to your dataset
dataset = pd.read_csv('ab.csv', encoding='latin1')

# Preprocess the dataset
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dataset['a'])
total_words = len(tokenizer.word_index) + 1

# Tokenize and pad the sequences
input_sequences = []
for line in dataset['a']:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]
        input_sequences.append(n_gram_sequence)

max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

# Separate input and target sequences
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

# Create and compile the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length - 1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# Train the model with a specified number of epochs
model.fit(X, y, epochs=10)  # You can specify the number of epochs here

# Save the trained model to a pkl file
with open('model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

# Initialize a spell checker
spell = SpellChecker()

# Load the trained model from the pkl file
with open('model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

# Recreate the tokenizer from the training data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dataset['a'])

def generate_response(user_input):
    # Preprocess user input
    user_input = user_input.lower()  # Convert to lowercase
    user_input = spell.correction(user_input)  # Correct spelling
    input_sequence = tokenizer.texts_to_sequences([user_input])[0]
    input_sequence = pad_sequences([input_sequence], maxlen=max_sequence_length - 1, padding='pre')

    # Generate a response using the trained model
    response_sequence = []
    for _ in range(max_sequence_length - 1):
        predicted_word_index = model.predict(input_sequence, verbose=0).argmax()
        predicted_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                predicted_word = word
                break
        response_sequence.append(predicted_word)
        # Update the input_sequence for the next iteration
        input_sequence[0][-1] = predicted_word_index

    response = ' '.join(response_sequence)
    return response

if __name__ == "__main__":
    while True:
        user_input = input("You: ")
        bot_response = generate_response(user_input)
        print(f"Bot: {bot_response}")


Epoch 1/10
646/646 [==============================] - 14s 18ms/step - loss: 6.3370
Epoch 2/10
646/646 [==============================] - 12s 18ms/step - loss: 5.7879
Epoch 3/10
646/646 [==============================] - 12s 18ms/step - loss: 5.3413
Epoch 4/10
646/646 [==============================] - 12s 18ms/step - loss: 4.9780
Epoch 5/10
646/646 [==============================] - 12s 18ms/step - loss: 4.6638
Epoch 6/10
646/646 [==============================] - 12s 18ms/step - loss: 4.3907
Epoch 7/10
646/646 [==============================] - 12s 18ms/step - loss: 4.1381
Epoch 8/10
646/646 [==============================] - 12s 18ms/step - loss: 3.8999
Epoch 9/10
646/646 [==============================] - 12s 18ms/step - loss: 3.6785
Epoch 10/10
646/646 [==============================] - 12s 18ms/step - loss: 3.4626
You: hi
Bot: i think you can you can you can you can you can you can you can you can
You: hello
Bot: a good luck for you can you can you can you can you can you can you 

AttributeError: 'NoneType' object has no attribute 'lower'

In [9]:
import pickle
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from spellchecker import SpellChecker

# Load your dialog dataset
# Replace 'ab.csv' with the actual path to your dataset
dataset = pd.read_csv('ab.csv', encoding='latin1')

# Preprocess the dataset
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dataset['a'])
total_words = len(tokenizer.word_index) + 1

# Tokenize and pad the sequences
input_sequences = []
for line in dataset['a']:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]
        input_sequences.append(n_gram_sequence)

max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

# Separate input and target sequences
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

# Create and compile the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length - 1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# Train the model with a specified number of epochs
model.fit(X, y, epochs=100)  # You can specify the number of epochs here

# Save the trained model to a pkl file
with open('model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

# Save the tokenizer to a pkl file
with open('tokenizer.pkl', 'wb') as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)

# Initialize a spell checker
spell = SpellChecker()

# Load the trained model from the pkl file
with open('model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

# Load the tokenizer from the pkl file
with open('tokenizer.pkl', 'rb') as tokenizer_file:
    tokenizer = pickle.load(tokenizer_file)

def generate_response(user_input):
    # Preprocess user input
    user_input = user_input.lower()  # Convert to lowercase
    user_input = spell.correction(user_input)  # Correct spelling
    input_sequence = tokenizer.texts_to_sequences([user_input])[0]
    input_sequence = pad_sequences([input_sequence], maxlen=max_sequence_length - 1, padding='pre')

    # Generate a response using the trained model
    response_sequence = []
    for _ in range(max_sequence_length - 1):
        predicted_word_index = model.predict(input_sequence, verbose=0).argmax()
        predicted_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                predicted_word = word
                break
        response_sequence.append(predicted_word)
        # Update the input_sequence for the next iteration
        input_sequence[0][-1] = predicted_word_index

    response = ' '.join(response_sequence)
    return response

if __name__ == "__main":
    while True:
        user_input = input("You: ")
        bot_response = generate_response(user_input)
        print(f"Bot: {bot_response}")


Epoch 1/100
646/646 [==============================] - 14s 19ms/step - loss: 6.3331
Epoch 2/100
646/646 [==============================] - 12s 18ms/step - loss: 5.8055
Epoch 3/100
646/646 [==============================] - 12s 18ms/step - loss: 5.3794
Epoch 4/100
646/646 [==============================] - 12s 19ms/step - loss: 5.0141
Epoch 5/100
646/646 [==============================] - 12s 19ms/step - loss: 4.6792
Epoch 6/100
646/646 [==============================] - 12s 19ms/step - loss: 4.3891
Epoch 7/100
646/646 [==============================] - 12s 19ms/step - loss: 4.1261
Epoch 8/100
646/646 [==============================] - 12s 18ms/step - loss: 3.8843
Epoch 9/100
646/646 [==============================] - 12s 18ms/step - loss: 3.6573
Epoch 10/100
646/646 [==============================] - 12s 19ms/step - loss: 3.4473
Epoch 11/100
646/646 [==============================] - 12s 19ms/step - loss: 3.2453
Epoch 12/100
646/646 [==============================] - 12s 19ms/step - lo

646/646 [==============================] - 12s 19ms/step - loss: 0.7937
Epoch 98/100
646/646 [==============================] - 12s 19ms/step - loss: 0.7943
Epoch 99/100
646/646 [==============================] - 12s 19ms/step - loss: 0.7938
Epoch 100/100
646/646 [==============================] - 12s 19ms/step - loss: 0.7922


In [3]:
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from spellchecker import SpellChecker

# Load the trained model from the pkl file
with open('model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

# Load the tokenizer from the pkl file
with open('tokenizer.pkl', 'rb') as tokenizer_file:
    tokenizer = pickle.load(tokenizer_file)

# Initialize a spell checker
spell = SpellChecker()

def generate_response(user_input):
    # Preprocess user input
    user_input = user_input.lower()  # Convert to lowercase
    user_input = spell.correction(user_input)  # Correct spelling
    input_sequence = tokenizer.texts_to_sequences([user_input])[0]
    input_sequence = pad_sequences([input_sequence], maxlen=model.input_shape[1], padding='pre')

    # Generate a response using the trained model
    response_sequence = []
    for _ in range(model.input_shape[1]):
        predicted_word_index = model.predict(input_sequence, verbose=0).argmax()
        predicted_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                predicted_word = word
                break
        response_sequence.append(predicted_word)
        # Update the input_sequence for the next iteration
        input_sequence[0][-1] = predicted_word_index

    response = ' '.join(response_sequence)
    return response

if __name__ == "__main__":
    while True:
        user_input = input("You: ")
        bot_response = generate_response(user_input)
        print(f"Bot: {bot_response}")


You: hi
Bot: there are you don't you don't you don't you don't you don't you don't you don't you don't
You: hello
Bot: we can we can we can we can we can we can we can we can we can
You: hi, how are you doing?


AttributeError: 'NoneType' object has no attribute 'lower'

In [6]:
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from spellchecker import SpellChecker

# Load the trained model from the pkl file
with open('model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

# Load the tokenizer from the pkl file
with open('tokenizer.pkl', 'rb') as tokenizer_file:
    tokenizer = pickle.load(tokenizer_file)

# Initialize a spell checker
spell = SpellChecker()

def generate_response(user_input):
    # Check if user input is not empty
    if not user_input:
        return "Bot: Please enter a message."

    # Preprocess user input
    user_input = user_input.lower()  # Convert to lowercase
    user_input = spell.correction(user_input)  # Correct spelling
    input_sequence = tokenizer.texts_to_sequences([user_input])[0]
    input_sequence = pad_sequences([input_sequence], maxlen=model.input_shape[1], padding='pre')

    # Generate a response using the trained model
    response_sequence = []
    for _ in range(model.input_shape[1]):
        predicted_word_index = model.predict(input_sequence, verbose=0).argmax()
        predicted_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                predicted_word = word
                break
        response_sequence.append(predicted_word)
        # Update the input_sequence for the next iteration
        input_sequence[0][-1] = predicted_word_index

    response = ' '.join(response_sequence)
    return "Bot: " + response

if __name__ == "__main__":
    while True:
        user_input = input("You: ")
        bot_response = generate_response(user_input)
        print(bot_response)


You: k7
Bot: there are you don't you don't you don't you don't you don't you don't you don't you don't
You: vimal
Bot: there are you don't you don't you don't you don't you don't you don't you don't you don't
You: sairam
Bot: there are you don't you don't you don't you don't you don't you don't you don't you don't
You: gokul
Bot: there are you don't you don't you don't you don't you don't you don't you don't you don't
You: vel
Bot: there are you don't you don't you don't you don't you don't you don't you don't you don't
You: howareyou


AttributeError: 'NoneType' object has no attribute 'lower'

In [7]:
def generate_response(user_input):
    # Check if user input is not empty
    if not user_input:
        return "Bot: Please enter a message."

    # Preprocess user input
    user_input = user_input.lower()  # Convert to lowercase
    corrected_input = spell.correction(user_input)  # Correct spelling

    # Check if the correction result is not None
    if corrected_input is None:
        return "Bot: I'm sorry, I couldn't understand your input."

    input_sequence = tokenizer.texts_to_sequences([corrected_input])[0]
    input_sequence = pad_sequences([input_sequence], maxlen=model.input_shape[1], padding='pre')

    # Generate a response using the trained model
    response_sequence = []
    for _ in range(model.input_shape[1]):
        predicted_word_index = model.predict(input_sequence, verbose=0).argmax()
        predicted_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                predicted_word = word
                break
        response_sequence.append(predicted_word)
        # Update the input_sequence for the next iteration
        input_sequence[0][-1] = predicted_word_index

    response = ' '.join(response_sequence)
    return "Bot: " + response


In [ ]:
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from spellchecker import SpellChecker

# Load the trained model from the pkl file
with open('model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

# Load the tokenizer from the pkl file
with open('tokenizer.pkl', 'rb') as tokenizer_file:
    tokenizer = pickle.load(tokenizer_file)

# Initialize a spell checker
spell = SpellChecker()

def generate_response(user_input):
    # Check if user input is not empty
    if not user_input:
        return "Bot: Please enter a message."

    # Preprocess user input
    user_input = user_input.lower()  # Convert to lowercase
    corrected_input = spell.correction(user_input)  # Correct spelling

    # Check if the correction result is not None
    if corrected_input is None:
        return "Bot: I'm sorry, I couldn't understand your input."

    input_sequence = tokenizer.texts_to_sequences([corrected_input])[0]
    input_sequence = pad_sequences([input_sequence], maxlen=model.input_shape[1], padding='pre')

    # Generate a response using the trained model
    response_sequence = []
    for _ in range(model.input_shape[1]):
        predicted_word_index = model.predict(input_sequence, verbose=0).argmax()
        predicted_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                predicted_word = word
                break
        response_sequence.append(predicted_word)
        # Update the input_sequence for the next iteration
        input_sequence[0][-1] = predicted_word_index

    response = ' '.join(response_sequence)
    return "Bot: " + response

if __name__ == "__main__":
    print("Bot: Hello! How can I assist you today?")
    
    while True:
        user_input = input("You: ")
        bot_response = generate_response(user_input)
        print(bot_response)


Bot: Hello! How can I assist you today?
You: hii
Bot: doctor you should take me too bad thanks i don't you should take me too bad thanks i don't you should take me too bad thanks i don't you should take
You: bye
Bot: even though i don't you should take me too bad thanks i don't you should take me too bad thanks i don't you should take me too bad thanks i don't
You: goodnight
Bot: what do you should take me too bad thanks i don't you should take me too bad thanks i don't you should take me too bad thanks i don't you should


In [14]:
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from spellchecker import SpellChecker

# Load your dialog dataset from 'dialog.txt'
with open('dialogs.txt', 'r', encoding='utf-8') as file:
    dialog_data = file.read().splitlines()

# Preprocess the dataset
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dialog_data)
total_words = len(tokenizer.word_index) + 1

# Tokenize and pad the sequences
input_sequences = []
for line in dialog_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]
        input_sequences.append(n_gram_sequence)

max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

# Separate input and target sequences
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

# Create and compile the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length - 1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# Train the model with a specified number of epochs
model.fit(X, y, epochs=50)  # You can specify the number of epochs here

# Save the trained model to a pkl file
with open('model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

# Save the tokenizer to a pkl file
with open('tokenizer.pkl', 'wb') as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)

# Initialize a spell checker
spell = SpellChecker()


# Load the trained model from the pkl file
with open('model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

# Load the tokenizer from the pkl file
with open('tokenizer.pkl', 'rb') as tokenizer_file:
    tokenizer = pickle.load(tokenizer_file)

def generate_response(user_input):
    # Preprocess user input
    user_input = user_input.lower()  # Convert to lowercase
    user_input = spell.correction(user_input)  # Correct spelling
    input_sequence = tokenizer.texts_to_sequences([user_input])[0]
    input_sequence = pad_sequences([input_sequence], maxlen=max_sequence_length - 1, padding='pre')

    # Generate a response using the trained model
    response_sequence = []
    for _ in range(max_sequence_length - 1):
        predicted_word_index = model.predict(input_sequence, verbose=0).argmax()
        predicted_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                predicted_word = word
                break
        response_sequence.append(predicted_word)
        # Update the input_sequence for the next iteration
        input_sequence[0][-1] = predicted_word_index

    response = ' '.join(response_sequence)
    return response

if __name__ == "__main__":
    while True:
        user_input = input("You: ")
        bot_response = generate_response(user_input)
        print(f"Bot: {bot_response}")


Epoch 1/50
1387/1387 [==============================] - 40s 27ms/step - loss: 6.0202
Epoch 2/50
1387/1387 [==============================] - 37s 27ms/step - loss: 5.1943
Epoch 3/50
1387/1387 [==============================] - 38s 27ms/step - loss: 4.6459
Epoch 4/50
1387/1387 [==============================] - 38s 27ms/step - loss: 4.2044
Epoch 5/50
1387/1387 [==============================] - 38s 27ms/step - loss: 3.8243
Epoch 6/50
1387/1387 [==============================] - 38s 27ms/step - loss: 3.4820
Epoch 7/50
1387/1387 [==============================] - 38s 27ms/step - loss: 3.1748
Epoch 8/50
1387/1387 [==============================] - 38s 27ms/step - loss: 2.8982
Epoch 9/50
1387/1387 [==============================] - 38s 27ms/step - loss: 2.6575
Epoch 10/50
1387/1387 [==============================] - 38s 27ms/step - loss: 2.4427
Epoch 11/50
1387/1387 [==============================] - 38s 27ms/step - loss: 2.2507
Epoch 12/50
1387/1387 [==============================] - 38s 27

AttributeError: 'NoneType' object has no attribute 'lower'